In [ ]:
#@title Imports
import torch
import random
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
from sklearn.svm import SVR
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.utils import resample
# import matplotlib.pyplot as plt
# import matplotlib.lines as mlines
from scipy.stats import spearmanr
from sklearn.metrics import ndcg_score
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, normalize
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.utils import resample
# import matplotlib.pyplot as plt
# import matplotlib.lines as mlines
# import matplotlib.ticker as ticker
from scipy.stats import spearmanr
from scipy.stats import pearsonr
from sklearn.metrics import ndcg_score
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR, SVC
from sklearn.linear_model import Ridge
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments, AutoTokenizer, AutoModelForMaskedLM, DataCollatorForLanguageModeling
# from datasets import Dataset
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score, recall_score, roc_auc_score, accuracy_score
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#title Load sequences from csv
df = pd.read_csv('./drive/MyDrive/LazBF_sequences.csv')
LazBF_sequences = df['sequences'].tolist()
LazBF_labels = df['labels'].tolist()

df = pd.read_csv('./drive/MyDrive/LazBF_sample.csv')
LazBF_sample = df['sequences'].tolist()
LazBF_sample_labels = df['labels'].tolist()

df = pd.read_csv('./drive/MyDrive/LazDEF_sequences.csv')
LazDEF_sequences = df['sequences'].tolist()
LazDEF_labels = df['labels'].tolist()

df = pd.read_csv('./drive/MyDrive/LazDEF_sample.csv')
LazDEF_sample = df['sequences'].tolist()
LazDEF_sample_labels = df['labels'].tolist()

#title Load Embs
lazbf_mlm_none = np.load("./drive/MyDrive/Embeddings/LazBF_mlm_none.npy")
lazdef_mlm_none = np.load("./drive/MyDrive/Embeddings/LazDEF_mlm_none.npy")

lazbf_mlm_lazbf = np.load("./drive/MyDrive/Embeddings/LazBF_mlm_LazBF.npy")
lazdef_mlm_lazbf = np.load("./drive/MyDrive/Embeddings/LazDEF_mlm_LazBF.npy")

lazbf_mlm_lazdef = np.load("./drive/MyDrive/Embeddings/LazBF_mlm_LazDEF.npy")
lazdef_mlm_lazdef = np.load("./drive/MyDrive/Embeddings/LazDEF_mlm_LazDEF.npy")

In [ ]:
lazbf_mlm_PA = np.load("./drive/MyDrive/LazBF_mlm_PA.npy")
lazdef_mlm_PA = np.load("./drive/MyDrive/LazDEF_mlm_PA.npy")

In [ ]:
optX = lazbf_mlm_PA
opty = LazBF_sample_labels
print("Optimization highn")

model_list = [LogisticRegression, KNeighborsClassifier, RandomForestClassifier, AdaBoostClassifier, SVC, MLPClassifier]
model_names = ['Logistic regression', 'KNN classif', 'RF', 'AdaBoost', 'SVC', 'MLP']
parameters_list = [
    {'classifier__penalty': ("l1", "l2", "elasticnet", None), 'classifier__C':[0.1, 1, 5, 10]},
    {'classifier__n_neighbors': [5, 10, 25, 50], 'classifier__weights': ('uniform', 'distance')},
    {'classifier__n_estimators': [50, 100, 200, 500], 'classifier__criterion': ('gini', 'entropy', 'log_loss')},
    {'classifier__n_estimators': [50, 100, 200, 500], 'classifier__learning_rate': [0.1, 1, 5, 10]},
    {'classifier__kernel':('linear', 'rbf'), 'classifier__C':[0.1, 1, 5, 10]},
    {'classifier__hidden_layer_sizes': [50, 100, 500, 750, 1000], 'classifier__activation':('tanh', 'relu')},
]

for model, name, parameters in zip(model_list, model_names, parameters_list):
    steps = [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50)),
        ('classifier', model())
    ]
    pipeline = Pipeline(steps)
    grid_search = GridSearchCV(pipeline, parameters, scoring='accuracy')
    grid_search.fit(optX, opty)
    print(f'Best paramters for {name} {grid_search.best_params_}')
    print(f'Best model for {name} {grid_search.best_estimator_}')

Optimization highn


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and

Best paramters for Logistic regression {'classifier__C': 5, 'classifier__penalty': 'l2'}
Best model for Logistic regression Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', LogisticRegression(C=5))])
Best paramters for KNN classif {'classifier__n_neighbors': 50, 'classifier__weights': 'uniform'}
Best model for KNN classif Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', KNeighborsClassifier(n_neighbors=50))])
Best paramters for RF {'classifier__criterion': 'entropy', 'classifier__n_estimators': 500}
Best model for RF Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier',
                 RandomForestClassifier(criterion='entropy',
                                        n_estimators=500))])
Best paramters for AdaBoost {'classifier__learning_rate': 1, 'classifier__n_estimators': 500}
Best model for AdaBoost Pipeline(st

In [ ]:
optX = lazdef_mlm_PA
opty = LazDEF_sample_labels
print("Optimization highn")

model_list = [LogisticRegression, KNeighborsClassifier, RandomForestClassifier, AdaBoostClassifier, SVC, MLPClassifier]
model_names = ['Logistic regression', 'KNN classif', 'RF', 'AdaBoost', 'SVC', 'MLP']
parameters_list = [
    {'classifier__penalty': ("l1", "l2", "elasticnet", None), 'classifier__C':[0.1, 1, 5, 10]},
    {'classifier__n_neighbors': [5, 10, 25, 50], 'classifier__weights': ('uniform', 'distance')},
    {'classifier__n_estimators': [50, 100, 200, 500], 'classifier__criterion': ('gini', 'entropy', 'log_loss')},
    {'classifier__n_estimators': [50, 100, 200, 500], 'classifier__learning_rate': [0.1, 1, 5, 10]},
    {'classifier__kernel':('linear', 'rbf'), 'classifier__C':[0.1, 1, 5, 10]},
    {'classifier__hidden_layer_sizes': [50, 100, 500, 750, 1000], 'classifier__activation':('tanh', 'relu')},
]

for model, name, parameters in zip(model_list, model_names, parameters_list):
    steps = [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50)),
        ('classifier', model())
    ]
    pipeline = Pipeline(steps)
    grid_search = GridSearchCV(pipeline, parameters, scoring='accuracy')
    grid_search.fit(optX, opty)
    print(f'Best paramters for {name} {grid_search.best_params_}')
    print(f'Best model for {name} {grid_search.best_estimator_}')

Optimization highn


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and

Best paramters for Logistic regression {'classifier__C': 1, 'classifier__penalty': 'l2'}
Best model for Logistic regression Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', LogisticRegression(C=1))])
Best paramters for KNN classif {'classifier__n_neighbors': 50, 'classifier__weights': 'uniform'}
Best model for KNN classif Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', KNeighborsClassifier(n_neighbors=50))])
Best paramters for RF {'classifier__criterion': 'log_loss', 'classifier__n_estimators': 500}
Best model for RF Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier',
                 RandomForestClassifier(criterion='log_loss',
                                        n_estimators=500))])


In [ ]:
optX = lazbf_mlm_PA
opty = LazBF_sample_labels
print("Optimization LOWN")

model_list = [LogisticRegression, KNeighborsClassifier, RandomForestClassifier, AdaBoostClassifier, SVC, MLPClassifier]
model_names = ['Logistic regression', 'KNN classif', 'RF', 'AdaBoost', 'SVC', 'MLP']
parameters_list = [
    {'classifier__penalty': ("l1", "l2", "elasticnet", None), 'classifier__C':[0.1, 1, 5, 10]},
    {'classifier__n_neighbors': [5, 10, 25, 50], 'classifier__weights': ('uniform', 'distance')},
    {'classifier__n_estimators': [50, 100, 200, 500], 'classifier__criterion': ('gini', 'entropy', 'log_loss')},
    {'classifier__n_estimators': [50, 100, 200, 500], 'classifier__learning_rate': [0.1, 1, 5, 10]},
    {'classifier__kernel':('linear', 'rbf'), 'classifier__C':[0.1, 1, 5, 10]},
    {'classifier__hidden_layer_sizes': [50, 100, 500, 750, 1000], 'classifier__activation':('tanh', 'relu')},
]

for model, name, parameters in zip(model_list, model_names, parameters_list):

    random_subset_indices = random.sample(range(len(optX)), 100)
    random_optX = [optX[i] for i in random_subset_indices]
    random_opty = [opty[i] for i in random_subset_indices]

    steps = [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50)),
        ('classifier', model())
    ]
    pipeline = Pipeline(steps)
    grid_search = GridSearchCV(pipeline, parameters, scoring='accuracy')
    grid_search.fit(random_optX, random_opty)
    print(f'Best paramters for {name} {grid_search.best_params_}')
    print(f'Best model for {name} {grid_search.best_estimator_}')

Optimization LOWN


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):

Best paramters for Logistic regression {'classifier__C': 1, 'classifier__penalty': 'l2'}
Best model for Logistic regression Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', LogisticRegression(C=1))])
Best paramters for KNN classif {'classifier__n_neighbors': 10, 'classifier__weights': 'uniform'}
Best model for KNN classif Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', KNeighborsClassifier(n_neighbors=10))])
Best paramters for RF {'classifier__criterion': 'entropy', 'classifier__n_estimators': 50}
Best model for RF Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier',
                 RandomForestClassifier(criterion='entropy', n_estimators=50))])
Best paramters for AdaBoost {'classifier__learning_rate': 1, 'classifier__n_estimators': 50}
Best model for AdaBoost Pipeline(steps=[('scaler', StandardScaler()), ('pca', 

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Best paramters for MLP {'classifier__activation': 'relu', 'classifier__hidden_layer_sizes': 750}
Best model for MLP Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', MLPClassifier(hidden_layer_sizes=750))])


In [ ]:
optX = lazdef_mlm_PA
opty = LazDEF_sample_labels
print("Optimization LOWN")

model_list = [LogisticRegression, KNeighborsClassifier, RandomForestClassifier, AdaBoostClassifier, SVC, MLPClassifier]
model_names = ['Logistic regression', 'KNN classif', 'RF', 'AdaBoost', 'SVC', 'MLP']
parameters_list = [
    {'classifier__penalty': ("l1", "l2", "elasticnet", None), 'classifier__C':[0.1, 1, 5, 10]},
    {'classifier__n_neighbors': [5, 10, 25, 50], 'classifier__weights': ('uniform', 'distance')},
    {'classifier__n_estimators': [50, 100, 200, 500], 'classifier__criterion': ('gini', 'entropy', 'log_loss')},
    {'classifier__n_estimators': [50, 100, 200, 500], 'classifier__learning_rate': [0.1, 1, 5, 10]},
    {'classifier__kernel':('linear', 'rbf'), 'classifier__C':[0.1, 1, 5, 10]},
    {'classifier__hidden_layer_sizes': [50, 100, 500, 750, 1000], 'classifier__activation':('tanh', 'relu')},
]

for model, name, parameters in zip(model_list, model_names, parameters_list):

    random_subset_indices = random.sample(range(len(optX)), 100)
    random_optX = [optX[i] for i in random_subset_indices]
    random_opty = [opty[i] for i in random_subset_indices]

    steps = [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50)),
        ('classifier', model())
    ]
    pipeline = Pipeline(steps)
    grid_search = GridSearchCV(pipeline, parameters, scoring='accuracy')
    grid_search.fit(random_optX, random_opty)
    print(f'Best paramters for {name} {grid_search.best_params_}')
    print(f'Best model for {name} {grid_search.best_estimator_}')

Optimization LOWN


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and

Best paramters for Logistic regression {'classifier__C': 1, 'classifier__penalty': 'l2'}
Best model for Logistic regression Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', LogisticRegression(C=1))])
Best paramters for KNN classif {'classifier__n_neighbors': 25, 'classifier__weights': 'uniform'}
Best model for KNN classif Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', KNeighborsClassifier(n_neighbors=25))])
Best paramters for RF {'classifier__criterion': 'gini', 'classifier__n_estimators': 200}
Best model for RF Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', RandomForestClassifier(n_estimators=200))])
Best paramters for AdaBoost {'classifier__learning_rate': 1, 'classifier__n_estimators': 200}
Best model for AdaBoost Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
               

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Best paramters for MLP {'classifier__activation': 'relu', 'classifier__hidden_layer_sizes': 500}
Best model for MLP Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', MLPClassifier(hidden_layer_sizes=500))])


---

In [ ]:
optX = lazbf_mlm_PA
opty = LazBF_sample_labels
print("Optimization MEDN")

model_list = [LogisticRegression, KNeighborsClassifier, RandomForestClassifier, AdaBoostClassifier, SVC, MLPClassifier]
model_names = ['Logistic regression', 'KNN classif', 'RF', 'AdaBoost', 'SVC', 'MLP']
parameters_list = [
    {'classifier__penalty': ("l1", "l2", "elasticnet", None), 'classifier__C':[0.1, 1, 5, 10]},
    {'classifier__n_neighbors': [5, 10, 25, 50], 'classifier__weights': ('uniform', 'distance')},
    {'classifier__n_estimators': [50, 100, 200, 500], 'classifier__criterion': ('gini', 'entropy', 'log_loss')},
    {'classifier__n_estimators': [50, 100, 200, 500], 'classifier__learning_rate': [0.1, 1, 5, 10]},
    {'classifier__kernel':('linear', 'rbf'), 'classifier__C':[0.1, 1, 5, 10]},
    {'classifier__hidden_layer_sizes': [50, 100, 500, 750, 1000], 'classifier__activation':('tanh', 'relu')},
]

for model, name, parameters in zip(model_list, model_names, parameters_list):

    random_subset_indices = random.sample(range(len(optX)), 1000)
    random_optX = [optX[i] for i in random_subset_indices]
    random_opty = [opty[i] for i in random_subset_indices]

    steps = [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50)),
        ('classifier', model())
    ]
    pipeline = Pipeline(steps)
    grid_search = GridSearchCV(pipeline, parameters, scoring='accuracy')
    grid_search.fit(random_optX, random_opty)
    print(f'Best paramters for {name} {grid_search.best_params_}')
    print(f'Best model for {name} {grid_search.best_estimator_}')

Optimization MEDN


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and

Best paramters for Logistic regression {'classifier__C': 1, 'classifier__penalty': 'l2'}
Best model for Logistic regression Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', LogisticRegression(C=1))])
Best paramters for KNN classif {'classifier__n_neighbors': 10, 'classifier__weights': 'uniform'}
Best model for KNN classif Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', KNeighborsClassifier(n_neighbors=10))])
Best paramters for RF {'classifier__criterion': 'entropy', 'classifier__n_estimators': 500}
Best model for RF Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier',
                 RandomForestClassifier(criterion='entropy',
                                        n_estimators=500))])
Best paramters for AdaBoost {'classifier__learning_rate': 0.1, 'classifier__n_estimators': 200}
Best model for AdaBoost Pipeline(

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Best paramters for MLP {'classifier__activation': 'tanh', 'classifier__hidden_layer_sizes': 500}
Best model for MLP Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier',
                 MLPClassifier(activation='tanh', hidden_layer_sizes=500))])


In [ ]:
optX = lazdef_mlm_PA
opty = LazDEF_sample_labels
print("Optimization MEDN")

model_list = [LogisticRegression, KNeighborsClassifier, RandomForestClassifier, AdaBoostClassifier, SVC, MLPClassifier]
model_names = ['Logistic regression', 'KNN classif', 'RF', 'AdaBoost', 'SVC', 'MLP']
parameters_list = [
    {'classifier__penalty': ("l1", "l2", "elasticnet", None), 'classifier__C':[0.1, 1, 5, 10]},
    {'classifier__n_neighbors': [5, 10, 25, 50], 'classifier__weights': ('uniform', 'distance')},
    {'classifier__n_estimators': [50, 100, 200, 500], 'classifier__criterion': ('gini', 'entropy', 'log_loss')},
    {'classifier__n_estimators': [50, 100, 200, 500], 'classifier__learning_rate': [0.1, 1, 5, 10]},
    {'classifier__kernel':('linear', 'rbf'), 'classifier__C':[0.1, 1, 5, 10]},
    {'classifier__hidden_layer_sizes': [50, 100, 500, 750, 1000], 'classifier__activation':('tanh', 'relu')},
]

for model, name, parameters in zip(model_list, model_names, parameters_list):

    random_subset_indices = random.sample(range(len(optX)), 1000)
    random_optX = [optX[i] for i in random_subset_indices]
    random_opty = [opty[i] for i in random_subset_indices]

    steps = [
        ('scaler', StandardScaler()),
        ('pca', PCA(n_components=50)),
        ('classifier', model())
    ]
    pipeline = Pipeline(steps)
    grid_search = GridSearchCV(pipeline, parameters, scoring='accuracy')
    grid_search.fit(random_optX, random_opty)
    print(f'Best paramters for {name} {grid_search.best_params_}')
    print(f'Best model for {name} {grid_search.best_estimator_}')

Optimization MEDN


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1181: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):

Best paramters for Logistic regression {'classifier__C': 0.1, 'classifier__penalty': 'l2'}
Best model for Logistic regression Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', LogisticRegression(C=0.1))])
Best paramters for KNN classif {'classifier__n_neighbors': 25, 'classifier__weights': 'distance'}
Best model for KNN classif Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier',
                 KNeighborsClassifier(n_neighbors=25, weights='distance'))])
Best paramters for RF {'classifier__criterion': 'entropy', 'classifier__n_estimators': 200}
Best model for RF Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier',
                 RandomForestClassifier(criterion='entropy',
                                        n_estimators=200))])
Best paramters for AdaBoost {'classifier__learning_rate': 1, 'classifier__n_estimators'

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

Best paramters for MLP {'classifier__activation': 'relu', 'classifier__hidden_layer_sizes': 1000}
Best model for MLP Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=50)),
                ('classifier', MLPClassifier(hidden_layer_sizes=1000))])
